In this notebook, we will analyse the simulated data using EasyScience. As EasyScience is still under development, there are some quirks. The only notable one here is that we need to install a particular branch of it, which can be done using the following command:

```
pip install git+https://github.com/EasyScience/EasyScience.git@numpy2
```

In [ ]:
import scipp as sc
import plopp as pp
import scippneutron as scn
import scippnexus as snx
import h5py
from pathlib import Path
import numpy as np
from easyscience.Objects.new_variable import Parameter
from easyscience.Objects.ObjectClasses import BaseObj
from easyscience.fitting import Fitter
import matplotlib.pyplot as plt
from ess.spectroscopy.indirect import bifrost
from bifrost2409.config import POOCH_DATA_DIR, INTERIM_DATA_DIR


In [2]:
datafile = "20240914/BIFROST_20240914T053723.h5"

We will load the data just like before

In [3]:
targets = ['energy_momentum_events']
target_files = {target: INTERIM_DATA_DIR / f'{Path(datafile).stem}_{target}.h5' for target in targets}
if all(file.exists() for file in target_files.values()):
    from scipp.io import load_hdf5
    objects = {target: load_hdf5(file) for target, file in target_files.items()}
else:
    data = bifrost(POOCH_DATA_DIR / datafile, is_simulated=True)
    objects = {target: data[target] for target in targets}
    for target in targets:
        objects[target].save_hdf5(target_files[target])


In [4]:
energy_momentum_events = objects['energy_momentum_events']

Let us have a look at the data again, before we decide how to analyse it

In [5]:
def hist_E_plane(events, q_x_range, q_bins, e_bins):
    # Creates a histogram of intensity as function of energy transfer and Q_x
    # events: scipp dataset with the events to be binned
    # q_x_range: scipp variable with the range of Q_x values to bin in
    # q_bins: The number of bins in q
    # e_bins: The number of bins in energy transfer
    
    a = events.bin(table_momentum_x=q_x_range)
    # Remove coordinates and event coordinates that we're not using:
    for coord in ('a3', 'a4', 'detector_number', 'final_energy'):
        del a.coords[coord]
    for coord in ('event_time_offset', 'event_time_zero', 'frame_time', 'incident_energy', 'lab_momentum_x', 'lab_momentum_z'):
        del a.bins.coords[coord]
    # drop the non-energy_transfer dimensions before binning in Q
    for dim in ('setting', 'event_id'):
        a = a.bins.concat(dim)
    return a.bin(energy_transfer=e_bins, table_momentum_z=q_bins).hist()['table_momentum_x', 0]
 

In [6]:
astar = 2 * np.pi / 6.56162

In [ ]:
my_events=hist_E_plane(energy_momentum_events, sc.array(values=[2 * astar - 0.2,  2 * astar + 0.2], dims=['table_momentum_x'], unit='1/angstrom'), 200, 50)
my_events.plot(norm='log')

We want to fit this data by taking 1-dimensional "cuts" at various constant values of Q, and determine the position of the peak from the phonon. We will fit these peak positions to a Gaussian.
Taking cuts of the full data set is inefficient, so we first prepare the data by removing the coordinates that we don't need. 

In [8]:
def prepare_data_for_cut(events):
    # Prepare the data for a cut in Q, by removing the coordinates that we're not using
    q_x_range=sc.array(values=[2 * astar - 0.2,  2 * astar + 0.2], dims=['table_momentum_x'], unit='1/angstrom')
    q_z_range=sc.array(values=[-2 * astar,  2 * astar], dims=['table_momentum_z'], unit='1/angstrom')
    a = events.bin(table_momentum_x=q_x_range,table_momentum_z=q_z_range)
    # Remove coordinates and event coordinates that we're not using:
    for coord in ('a3', 'a4', 'detector_number', 'final_energy'):
        del a.coords[coord]
    for coord in ('event_time_offset', 'event_time_zero', 'frame_time', 'incident_energy', 'lab_momentum_x', 'lab_momentum_z'):
        del a.bins.coords[coord]
    # drop the non-energy_transfer dimensions before binning in Q
    for dim in ('setting', 'event_id'):
        a = a.bins.concat(dim)    
    return a
    
prepared_data=prepare_data_for_cut(energy_momentum_events)

We now define a function to make cuts at constant Q. Many of the bins have zero count, and in these cases, the variance is also zero. A normal chi square fit cannot handle this, so we artifically add 1 to the variance. It is also possible to fit the data using the Poisson likelihood instead, but we save that for another day. 

In [ ]:
def const_Q_cut(events, q_z_value,q_z_width,E_bins,E_min,E_max):
    # Creates a histogram of intensity as function of energy transfer for a given Q_z value
    # events: scipp dataset with the events to be binned
    # q_z_value: The Q_z value to cut at
    # q_z_width: The width of the cut
    # E_bins: The number of bins in energy transfer
    # E_min: The minimum energy transfer as a scipp variable
    # E_max: The maximum energy transfer as a scipp variable
    q_z_range=sc.array(values=[q_z_value.value - q_z_width.value/2, q_z_value.value + q_z_width.value/2], dims=['table_momentum_z'],unit='1/angstrom')
    a = events.bin(table_momentum_z=q_z_range)
    a=a.bin(energy_transfer=E_bins).hist()

    a = a.assign_coords(energy_transfer=sc.midpoints(a.coords['energy_transfer']))
    a=a['table_momentum_x', 0]['table_momentum_z', 0]    
    a.variances = a.values+1.0

    a=a['energy_transfer',E_min:E_max]

    return a

#Test the function. Notice how we define units using scipp.
Q_cut_center = 1.0 * sc.Unit('1/angstrom')
Q_cut_width = 0.1 * sc.Unit('1/angstrom')
E_bins = 201
E_min = -0.1 * sc.Unit('meV')
E_max = 2 * sc.Unit('meV')
mycut=const_Q_cut(prepared_data,Q_cut_center,Q_cut_width,E_bins,E_min,E_max)
mycut.plot(norm='log')


Let us assume that the data can be fitted with a Gaussian plus a background.

In [10]:

# Function to calculate the Gaussian + background model for fitting. 
# When using EasyScience, you don't give the input parameters to the function.
def gaussian_model(E: np.ndarray) -> np.ndarray:
    y= A.value * np.exp(-((E - E0.value) ** 2) / (2 * sigma.value ** 2)) + B.value
    return y

In [ ]:

# Let us first do a single cut and fit to see how the fitting works
# Create a cut
Q_cut_center = 1.0* sc.Unit('1/angstrom') 
Q_cut_width = 0.1* sc.Unit('1/angstrom')
E_bins=201
E_min=1.1*sc.Unit('meV')
E_max=2*sc.Unit('meV')

this_data=const_Q_cut(prepared_data,Q_cut_center,Q_cut_width,201,E_min,E_max)

# For EasyScience we extract the raw values and errors
E_values=this_data.coords['energy_transfer'].values
intensity_cut=this_data.values
intensity_error_cut=np.sqrt(this_data.variances)

# Define the Gaussian parameters to fit (A = amplitude, E0 = center, sigma = width, B = background)
# We give sensible start values for all parameters
A =     Parameter(name='A',     value=np.max(intensity_cut),              fixed=False,min=0)
E0 =    Parameter(name='E0',    value=E_values[np.argmax(intensity_cut)], fixed=False)
sigma = Parameter(name='sigma', value=0.05,                               fixed=False,min=0)
B =     Parameter(name='B',     value=np.min(intensity_cut),              fixed=False)

# Create an EasyScience Base Object, containing the Gaussian function and its parameters. 
gaussian = BaseObj(name='gaussian', A=A, E0=E0, sigma=sigma, B=B)

# Create the Fitter object
fitter = Fitter(gaussian, gaussian_model)

# Fit the data for this cut. Here, weights are the inverse of the errors in order to emphasize the peak more than the background
res = fitter.fit(x=E_values, y=intensity_cut,weights=1/intensity_error_cut)

# Show the fit results
print(f"Q_cut_center: {round(Q_cut_center.value, 2)}, {A}, {E0}, {sigma}, {B}")

plt.figure(figsize=(5, 5))
plt.errorbar(E_values, intensity_cut, intensity_error_cut, label='Data',marker='o',markerfacecolor='w',linestyle='None')
E_values_fit = np.linspace(E_values[0], E_values[-1], 1000)
plt.plot(E_values_fit, gaussian_model(E_values_fit), label='Fit')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:

# We are now ready to loop over all Q cuts and fit the data for each cut
# Define the number of cuts and the Q range for the cuts
number_of_cuts=15
Q_z_start=0.1* sc.Unit('1/angstrom')
Q_z_end=1.6* sc.Unit('1/angstrom')

Q_cut_centers=sc.linspace('table_momentum_z', Q_z_start, Q_z_end, num=number_of_cuts) # Centers of the Q cuts

Q_cut_width = (Q_z_end-Q_z_start)/number_of_cuts  # Width of Q cuts

# Define the number of bins and the minimum energy transfer for the fits. To keep things simple, we want to avoid the elastic line
E_min=0.1*sc.Unit('meV')
E_max=2*sc.Unit('meV')

# Store the fit results for each Q cut
fit_results = [] 


# Loop over each Q cut
for Q_cut_center in Q_cut_centers:
    this_data=const_Q_cut(prepared_data,Q_cut_center,Q_cut_width,E_bins,E_min,E_max)
    E_values=this_data.coords['energy_transfer'].values
    intensity_cut=this_data.values
    intensity_error_cut=np.sqrt(this_data.variances)
   

    # Define the Gaussian parameters to fit (A = amplitude, E0 = center, sigma = width, B = background)
    A =     Parameter(name='A',     value=np.max(intensity_cut),              fixed=False,min=0)
    E0 =    Parameter(name='E0',    value=E_values[np.argmax(intensity_cut)], fixed=False)
    sigma = Parameter(name='sigma', value=0.05,                               fixed=False,min=0)
    B =     Parameter(name='B',     value=np.min(intensity_cut),              fixed=False)
    
    # Create the base object for the Gaussian
    gaussian = BaseObj(name='gaussian', A=A, E0=E0, sigma=sigma, B=B)
    # simple_gaussian = BaseObj(name='simple_gaussian', A=A)
    
    # Create the Fitter object
    fitter = Fitter(gaussian, gaussian_model)
    # fitter = Fitter(simple_gaussian, gaussian_model)
    
    # Fit the data for this cut
    res = fitter.fit(x=E_values, y=intensity_cut,weights=1/intensity_error_cut)
    
    # Save the fit parameters for this cut
    fit_results.append({
        'Q_cut_center': Q_cut_center,
        'A': A,
        'E0': E0,
        'sigma': sigma,
        'B': B,
        'intensity_cut': intensity_cut,
        'intensity_error_cut': intensity_error_cut,
        'E_values': E_values
    })

# Show the fit results
for result in fit_results:
    print(f"Q: {round(result['Q_cut_center'].value, 2)}, {result['A']}, {result['E0']}, {result['sigma']}, {result['B']}")

plt.figure(figsize=(15, 20))
for i, result in enumerate(fit_results):
# for i in range(1):

    Q_cut_center = result['Q_cut_center']
    A = result['A']
    E0 = result['E0']
    sigma = result['sigma']
    B = result['B']
    intensity_cut = result['intensity_cut']
    intensity_error_cut = result['intensity_error_cut']
    E_values = result['E_values']
    
    plt.subplot(int(np.ceil(number_of_cuts/3)), 3, i + 1)
    plt.errorbar(E_values, intensity_cut, intensity_error_cut, label='Data')
    plt.plot(E_values, gaussian_model(E_values), label='Fit')
    plt.title(f'Q = {Q_cut_center.value:.2f}')
    plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Extract the fitted parameters for each Q cut
Q_values =  np.array([result['Q_cut_center'].value for result in fit_results])  
E0_values = np.array([result['E0'].value for result in fit_results]) 
E0_errors = np.array([result['E0'].error for result in fit_results]) 
A_values = np.array([result['A'].value for result in fit_results]) 
A_errors = np.array([result['A'].error for result in fit_results]) 

# Define the parameter to fit 
K = Parameter(name='K', value=1.0, fixed=False,min=0)  

# Define dispersion
def dispersion_model(h: np.ndarray) -> np.ndarray:
    astar = 2 * np.pi / 6.56162
    return K.value*np.abs(np.sin(np.pi*h/2/astar))

# Create a BaseObj and fit the data
phonon_dispersion = BaseObj(name='phonon_dispersion', K=K)
fitter = Fitter(phonon_dispersion, dispersion_model)
res = fitter.fit(x=Q_values, y=E0_values, weights=1/E0_errors)

# Extract the fitted parameters
fitted_K = K

# Print the fitted parameters
print(f"Fitted K: {fitted_K}")

# Optional: Plot the fit against the data
import matplotlib.pyplot as plt

# Generate model values using the fitted parameters
h_fit = np.linspace(min(Q_values), max(Q_values), 100)
E_fit = dispersion_model(h_fit)

# Plot the data and the fitted model
plt.errorbar(Q_values/astar, E0_values, yerr=E0_errors, fmt='o', label='Data')
plt.plot(h_fit/astar, E_fit, label='Fitted Dispersion', color='red')
plt.xlabel('Q_z (R.L.U.)')
plt.ylabel('E (meV)')
plt.title('Fitted phonon dispersion')
plt.legend()
plt.ylim(0,1.6)
plt.show()

